In [4]:
import getpass
import os
import time
from dotenv import load_dotenv

# Load the .env file
_ = load_dotenv()

### Model

In [5]:
from agno.models.ollama import Ollama

sql_ollama_model=Ollama(id="qwen2.5-coder:7b")

## Database Connection

In [6]:
from agno.tools.sql import SQLTools
from sqlalchemy import create_engine

In [7]:
db_url = 'sqlite:///../data/shop.db'
engine = create_engine(db_url)

In [8]:
tool = SQLTools(db_engine=engine)
tool

<SQLTools name=sql_tools functions=['list_tables', 'describe_table', 'run_sql_query']>

In [9]:
tool.functions

OrderedDict([('list_tables',
              Function(name='list_tables', description=None, parameters={'type': 'object', 'properties': {}, 'required': []}, strict=None, entrypoint=<bound method SQLTools.list_tables of <SQLTools name=sql_tools functions=['list_tables', 'describe_table', 'run_sql_query']>>, sanitize_arguments=True, show_result=False, stop_after_tool_call=False, pre_hook=None, post_hook=None)),
             ('describe_table',
              Function(name='describe_table', description=None, parameters={'type': 'object', 'properties': {}, 'required': []}, strict=None, entrypoint=<bound method SQLTools.describe_table of <SQLTools name=sql_tools functions=['list_tables', 'describe_table', 'run_sql_query']>>, sanitize_arguments=True, show_result=False, stop_after_tool_call=False, pre_hook=None, post_hook=None)),
             ('run_sql_query',
              Function(name='run_sql_query', description=None, parameters={'type': 'object', 'properties': {}, 'required': []}, strict=Non

In [10]:
tool.list_tables()

'["Customers", "OrderDetails", "Orders", "Products"]'

In [11]:
tool.run_sql(sql="SELECT * FROM Customers", limit=1)

[{'CustomerID': 1,
  'FirstName': 'Juan',
  'LastName': 'Pérez',
  'Email': 'juan.perez@email.com',
  'Phone': '555-1234',
  'Address': 'Calle de la Rosa 12, Sevilla'}]

### QuerySql agent

In [ ]:
from agno.agent import Agent

querysql_agent = Agent(model=sql_ollama_model, name="sqlagent", tools=[tool.run_sql], show_tool_calls=True)

### Text 2 sql agent

In [3]:
from agno.agent import Agent

instructions="""
You are an expert in SQL. You can create sql queries from natural language following these schemas:

CREATE TABLE IF NOT EXISTS Customers(
        CustomerID INTEGER PRIMARY KEY AUTOINCREMENT,
        FirstName TEXT NOT NULL,
        LastName TEXT NOT NULL,
        Email TEXT UNIQUE NOT NULL,
        Phone TEXT,
        Address TEXT
);

-- Create Products table
CREATE TABLE IF NOT EXISTS Products (
        ProductID INTEGER PRIMARY KEY AUTOINCREMENT,
        Name TEXT NOT NULL,
        Description TEXT,
        Price REAL NOT NULL,
        StockQuantity INTEGER NOT NULL,
        Category TEXT NOT NULL
    );


-- Create Orders table
CREATE TABLE IF NOT EXISTS Orders (
        OrderID INTEGER PRIMARY KEY AUTOINCREMENT,
        CustomerID INTEGER NOT NULL,
        OrderDate DATE NOT NULL,
        TotalAmount REAL NOT NULL,
        FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
    );

-- Create OrderDetails table
CREATE TABLE IF NOT EXISTS OrderDetails (
        OrderDetailID INTEGER PRIMARY KEY AUTOINCREMENT,
        OrderID INTEGER NOT NULL,
        ProductID INTEGER NOT NULL,
        Quantity INTEGER NOT NULL,
        Subtotal REAL NOT NULL,
        FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
        FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
    );

Return only the sql query. Don't explain the query and don't put the sql query in ```sql\n```.
"""
text2sql_agent = Agent(model=sql_ollama_model, name="text2sql", instructions=instructions, show_tool_calls=True)

### Tests

In [4]:
def write_sql_query(query:str):
    """ Function to write sql query """
    start_time = time.perf_counter()

    response = text2sql_agent.run(query)

    end_time = time.perf_counter()
    
    latency = end_time - start_time
    
    print(f"Execution time: {latency} seconds")
    answer = response.content
    
    return answer
    

In [5]:
query = "¿Cuáles son los detalles de todos los clientes?"
write_sql_query(query) #SELECT * FROM Customers;

Execution time: 109.26334430000861 seconds


'SELECT * FROM Customers;'

In [6]:
query = "¿Cuáles son los productos disponibles en la categoría 'Women'?"
write_sql_query(query)#"SELECT * FROM Products WHERE Category = 'Women';",

Execution time: 6.971404300013091 seconds


"SELECT Name FROM Products WHERE Category = 'Women'"

In [7]:
query = "¿Cuál es el precio total de todos los pedidos realizados por un cliente específico con ID 5?"
write_sql_query(query)#"SELECT SUM(TotalAmount) FROM Orders WHERE CustomerID = 5;",

Execution time: 11.981810800003586 seconds


'SELECT SUM(TotalAmount) FROM Orders WHERE CustomerID = 5'

In [8]:
query = "¿Cuántos productos están en stock con una cantidad mayor a 20 unidades?"
write_sql_query(query)#"SELECT * FROM Products WHERE StockQuantity > 20;"

Execution time: 10.258081999985734 seconds


'SELECT COUNT(*) FROM Products WHERE StockQuantity > 20'

In [9]:
query = "¿Qué productos han sido comprados en el pedido con ID 3?"
write_sql_query(query)#"SELECT p.Name, od.Quantity, od.Subtotal FROM OrderDetails od INNER JOIN Products p ON od.ProductID = p.ProductID WHERE od.OrderID = 3;",

Execution time: 14.596958100009942 seconds


'SELECT P.ProductID, P.Name FROM Products AS P JOIN OrderDetails AS OD ON P.ProductID = OD.ProductID WHERE OD.OrderID = 3;'